<a href="https://colab.research.google.com/github/SankethGB/beginnerthough/blob/master/Mvie_recomendation2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import pandas as pd
import numpy as np
import torch
import torch.optim as optim
import torch.nn as nn
import torch.utils.data
from torch.autograd import Variable

In [0]:

users=pd.read_csv('/content/drive/My Drive/Machine Learning/Boltzmann_Machines/ml-1m/users.dat',sep='::',engine='python',header=None,encoding='latin-1')

In [0]:
movies=pd.read_csv('/content/drive/My Drive/Machine Learning/Boltzmann_Machines/ml-1m/movies.dat',sep='::',engine='python',header=None,encoding='latin-1')
ratings=pd.read_csv('/content/drive/My Drive/Machine Learning/Boltzmann_Machines/ml-1m/ratings.dat',sep='::',engine='python',header=None,encoding='latin-1')

In [0]:

training_set=pd.read_csv('/content/drive/My Drive/Machine Learning/Boltzmann_Machines/ml-100k/u1.base',delimiter='\t')

In [0]:
test_set=pd.read_csv('/content/drive/My Drive/Machine Learning/Boltzmann_Machines/ml-100k/u1.test',delimiter='\t')

In [0]:
training_set=np.array(training_set,dtype=int)
test_set=np.array(test_set,dtype=int)

In [0]:
max_users=int(max(max(training_set[:,0]),max(test_set[:,0])))
max_movies=int(max(max(training_set[:,1]),max(test_set[:,1])))

In [0]:
def convert(data):
  convdata=np.zeros((max_users+1)*(max_movies+1)).reshape(max_users+1,max_movies+1)
  for i in range(0,len(data)):
    convdata[data[i][0]][data[i][1]]=data[i][2]
  list(convdata)  
  return convdata

In [0]:
training_set=convert(training_set)
test_set=convert(test_set)

In [0]:
training_set=torch.FloatTensor(training_set)
test_set=torch.FloatTensor(test_set)

In [0]:
class SAE(nn.Module):
  def __init__(self):
    super(SAE,self).__init__()
    self.fc1=nn.Linear(max_movies+1,20)
    self.fc2=nn.Linear(20,10)
    self.fc3=nn.Linear(10,20)
    self.fc4=nn.Linear(20,max_movies+1)
    self.activation=nn.Sigmoid()
  def forward(self,x):
    x=self.activation(self.fc1(x))
    x=self.activation(self.fc2(x))
    x=self.activation(self.fc3(x))
    x=self.fc4(x)
    return x
    

  

In [0]:
sae=SAE()
criterion=nn.MSELoss()
optimiser=optim.RMSprop(sae.parameters(),lr=0.01,weight_decay=0.5)

In [0]:
nb_epochs=200
for epochs in range(1,nb_epochs+1):
  training_loss=0
  s=0.
  for users in range(max_users):
    x=Variable(training_set[users]).unsqueeze(0)
    target=x.clone()
    if torch.sum(target.data > 0) > 0:
      output=sae(x)
      target.requires_grad=False
      output[target ==0] = 0
      loss=criterion(output,target)
      mean_corrector=(max_movies + 1 )/ float(torch.sum(target.data>0)+1e-10)
      loss.backward()
      training_loss+=np.sqrt(loss.data * mean_corrector)
      s+=1.  
      optimiser.step()
  print("Epoch:"+str(epochs)+" Loss: "+str(training_loss/s))


Epoch:1 Loss: tensor(0.8870)
Epoch:2 Loss: tensor(0.8866)
Epoch:3 Loss: tensor(0.8853)
Epoch:4 Loss: tensor(0.8849)
Epoch:5 Loss: tensor(0.8852)
Epoch:6 Loss: tensor(0.8855)
Epoch:7 Loss: tensor(0.8843)
Epoch:8 Loss: tensor(0.8831)
Epoch:9 Loss: tensor(0.8835)
Epoch:10 Loss: tensor(0.8823)
Epoch:11 Loss: tensor(0.8817)
Epoch:12 Loss: tensor(0.8816)
Epoch:13 Loss: tensor(0.8808)
Epoch:14 Loss: tensor(0.8806)
Epoch:15 Loss: tensor(0.8806)
Epoch:16 Loss: tensor(0.8804)
Epoch:17 Loss: tensor(0.8804)
Epoch:18 Loss: tensor(0.8802)
Epoch:19 Loss: tensor(0.8807)
Epoch:20 Loss: tensor(0.8793)
Epoch:21 Loss: tensor(0.8791)
Epoch:22 Loss: tensor(0.8788)
Epoch:23 Loss: tensor(0.8776)
Epoch:24 Loss: tensor(0.8776)
Epoch:25 Loss: tensor(0.8780)
Epoch:26 Loss: tensor(0.8765)
Epoch:27 Loss: tensor(0.8767)
Epoch:28 Loss: tensor(0.8764)
Epoch:29 Loss: tensor(0.8758)
Epoch:30 Loss: tensor(0.8752)
Epoch:31 Loss: tensor(0.8749)
Epoch:32 Loss: tensor(0.8757)
Epoch:33 Loss: tensor(0.8748)
Epoch:34 Loss: tens

In [56]:
test_loss=0
s=0.
for users in range(max_users):
  x=Variable(training_set[users]).unsqueeze(0)
  target=Variable(test_set[users]).unsqueeze(0)
  if torch.sum(target.data > 0) > 0:
    output=sae(x)
    target.requires_grad=False
    output[target ==0] = 0
    loss=criterion(output,target)
    mean_corrector=(max_movies + 1 )/ float(torch.sum(target.data>0)+1e-10)
    test_loss+=np.sqrt(loss.data * mean_corrector)
    s+=1.  
print("Epoch:"+str(epochs)+" Loss: "+str(test_loss/s))

Epoch:200 Loss: tensor(0.9492)
